In [1]:
import pickle
import re

In [2]:
# load emails from pickle file where headers are parsed and separated from bodies 
# and emails sorted by time sent

with open("headers_processed.pkl", "rb") as handle:
    headers_srtd, mails_srtd = pickle.load(handle)

## Extract quoted text (from previous emails)

 - first, match the opening line of quoted text <br>
   (usually something like '_On Monday, 01 Jan 2000, Firstname Lastname wrote:_', although small differences making parsing difficult) <br>
   (some also have a line containing '_---Original Message---_' followed by a normal email header as the opening)
 - a quoted line is almost always indicated by leading '>' (sometimes also by leading '|') <br>
   vice versa also seems to be the case
   
   
 - for now, simply remove the quoted text; later try to create a new email instance from it or at least put as attachment
 - removal leads to a reduction of lines by 21% (8.6e6 to 6.7e6)

In [3]:
quote_opening_pattern =\
    re.compile(r"(?:^(?:[aA]t|[Oo]n)\s([\W\w]+)\s(?:([\w\.-]+@[\w\.-]+)|([a-zA-Z]+))\s(?:wrote|writes):)|(?:[^a-zA-Z]*[Oo]riginal message[^a-zA-Z]*)")

quote_pattern = re.compile(r"^(>|\|)[\W\w]+") # [ \t]


def remove_quoted_text(mail):
    for line in mail.split("\n"):
        if quote_opening_pattern.match(line) or quote_pattern.match(line):
            continue
        yield line
        
mails_pure = ["\n".join(remove_quoted_text(m)) for m in mails_srtd]

## Write processed emails to pickle file

In [4]:
with open("headers_and_bodies_processed.pkl", "wb") as handle:
    pickle.dump((headers_srtd, mails_pure), handle)

In [5]:
# quote_open_pattern =\
#     re.compile(r"(?:^(?:[aA]t|[Oo]n)\s([\W\w]+)\s(?:([\w\.-]+@[\w\.-]+)|([a-zA-Z]+))\s(?:wrote|writes):)|(?:[^a-zA-Z]*[Oo]riginal message[^a-zA-Z]*)")

# def read_quoted(mail_iter, open_line):
#     yield open_line
    
#     next_l = next(mail_iter)
#     yield next_l
    
#     while next_l.startswith(">") or next_l.startswith(" >") or (not next_l.strip()):
#         try:
#             next_l = next(mail_iter)
#             yield next_l
#         except StopIteration:
#             return
    
# def extract_quoted_text(mail_lines):
#     mail_iter = iter(mail_lines)
    
#     non_quoted_lines = []
    
#     quotes = []
    
#     for l in mail_iter:
#         if quote_open_pattern.match(l):
#             quoted_block = list(read_quoted(mail_iter, l))
#             quotes.append(quoted_block)
#         else:
#             non_quoted_lines.append(l)
            
#     return non_quoted_lines, quotes
    
# def extract_quoted_text(mail_lines):
#     mail_iter = iter(mail_lines)
    
#     quotes = []
#     non_quoted_lines = []
        
#     for l in mail_iter:
#         if quote_open_pattern.match(l):
#             quotes.append(l)
#         elif l.startswith(">") or l.startswith(" >"):
#             quotes.append(l)
#         else:
#             non_quoted_lines.append(l)
            
#     return non_quoted_lines, quotes